#### Ciencia de datos - Primavera 2024
##### Integrantes: Magdalena Cobb, Pedro García Vassallo, Marcos Olavarría
### Trabajo Práctico 4

# 1. LECTURA

In [55]:
#Paquetes y librerías a usar

#!pip install openpyxl
#!pip install statsmodels

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm 

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score 
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import RocCurveDisplay
#from sklearn.metrics import plot_roc_curve
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier

In [56]:
#Lectura del documento
ind_2024 = pd.read_excel('C:/Users/SAMSUNG/Documents/GitHub/CC408-T1-5/TP 4/usu_individual_T124.xlsx') 
indi_2004 = pd.read_stata('C:/Users/SAMSUNG/Documents/GitHub/CC408-T1-5/TP 4/Individual_t104.dta')
hog_2024 = pd.read_excel('C:/Users/SAMSUNG/Documents/GitHub/CC408-T1-5/TP 4/usu_hogar_T124.xlsx')
hog_2004 = pd.read_stata('C:/Users/SAMSUNG/Documents/GitHub/CC408-T1-5/TP 4/Individual_t104.dta')

# Maggie: C:/Users/magda/OneDrive/Documents/GitHub/CC408-T1-5/TP 4/
# Peter: 'C:/Users/SAMSUNG/Documents/GitHub/CC408-T1-5/TP4/Individual_t104.dta
# Marck: 'C:/Users/marcos.olavarria/Documents/GitHub/CC408/TP 4/usu_hogar_T124.xlsx'


# 2. LIMPIEZA

a. Eliminen todas las observaciones que no corresponden a los
aglomerados de Ciudad Autónoma de Buenos Aires o Gran Buenos
Aires, y unan ambos trimestres en una sola base.

In [57]:
#Filtramos las bases para tener solo Buenos Aires
ind_2024 = ind_2024[(ind_2024['AGLOMERADO'] == 33) | (ind_2024['AGLOMERADO'] == 32)]
ind_2004 = ind_2004[(ind_2004['aglomerado'] == 'Ciudad de Buenos Aires') | (ind_2004 ['aglomerado'] == 'Partidos del GBA' )]

ind_2004.columns = ind_2004.columns.str.upper()
ind_2024['IPCF'] =ind_2024['IPCF'].astype(int)

KeyError: 'aglomerado'

#### Se eliminan columnas antes de combinar bases

In [ ]:
#IDENTIFICO COLUMNAS QUE DIFIEREN ENTRE LAS BASES
col2004 = ind_2004.columns
col2024 = ind_2024.columns

# Encontrar columnas que son diferentes
columnas_df1 = set(col2004)
columnas_df2 = set(col2024)

# Columnas en df1 pero no en df2
diferentes_df1 = columnas_df1 - columnas_df2
# Columnas en df2 pero no en df1
diferentes_df2 = columnas_df2 - columnas_df1

# Comparar columnas comunes para ver diferencias en valores
diferencias_valores = {}
for col in columnas_df1.intersection(columnas_df2):
    if not ind_2004[col].equals(ind_2024[col]):
        diferencias_valores[col] = (ind_2004[col].tolist(), ind_2024[col].tolist())

print("Columnas solo en 2004:", diferentes_df1)
print("Columnas solo en 2024:", diferentes_df2)
#print("Diferencias en valores:", diferencias_valores)


Columnas solo en 2004: {'PJ2_1', 'IDIMPP', 'PJ3_1', 'PJ1_1'}
Columnas solo en 2024: {'IMPUTA', 'PONDIH', 'PONDII', 'PONDIIO', 'CH05'}


In [ ]:
#ELIMINO LAS COLUMNAS QUE DIFIEREN Y LAS QUE RESULTAN IRRELEVANTES
del_col1 = ['PJ3_1', 'IDIMPP', 'PJ2_1', 'PJ1_1']
del_col2 = ['IMPUTA', 'CH05', 'PONDII', 'PONDIH', 'PONDIIO']

ind_2004 =ind_2004.drop(columns = del_col1)
ind_2024 =ind_2024.drop(columns = del_col2)

# df_combined = df.drop(columns=['B', 'C'])

#### Se combinan las bases
b. Si hay observaciones con valores que no tienen sentido,
descártenlas (por ejemplo, ingresos y edades negativos). Expliquen
las decisiones tomadas.

In [ ]:
# Uno ambos trimestres en una sola base
df_combined = pd.concat([ind_2004, ind_2024], ignore_index=True, sort=False)
df_combined["ANO4"] = df_combined["ANO4"].astype(int)


In [ ]:
#Definición de algunas variables para uniformar diccionarios y unificar las bases
h04 = {
    "Varón": 1,
    "Mujer": 2
}

h06 = {
    'Menos de 1 año': 0,
    '98 y más años': 98
}

h07 = {
    "Unido": 1,
    "Casado": 2,
    "Separado o divorciado": 3,
    "Viudo": 4,
    "Soltero": 5
}

h08 =    {
    'Obra social (incluye PAMI)': 1,
    'Mutual/Prepaga/Servicio de emergencia': 2,
    'Planes y seguros públicos': 3,
    'No paga ni le descuentan': 4,
    'Ns./Nr.': 9,
    'Obra social y mutual/prepaga/servicio de emergencia': 12,
    'Obra social y planes y seguros públicos': 13,
    'Mutual/prepaga/servicio de emergencia/planes y seguros públi': 23,
    'Obra social, mutual/prepaga/servicio de emergencia y planes': 123
}

nivel_ed= {
    'Primaria Incompleta (incluye educación especial)': 1,
    'Primaria Completa': 2,
    'Secundaria Incompleta': 3,
    'Secundaria Completa': 4,
    'Superior Universitaria Incompleta': 5,
    'Superior Universitaria Completa': 6,
    'Sin instrucción': 7,
    'Ns/Nr': 9
}

estado = {
    'Entrevista individual no realizada (no respuesta al cuestion': 0,
    'Ocupado': 1,
    'Desocupado': 2,
    'Inactivo': 3,
    'Menor de 10 años': 4
}

cat_inac = {
    'Jubilado/pensionado': 1,
    'Rentista': 2,
    'Estudiante': 3,
    'Ama de casa': 4,
    'Menor de 6 años': 5,
    'Discapacitado': 6,
    'Otros': 7
}


componente = {
    'Servicio doméstico en hogares': 51
}

h15 = {
    "Sí": 1,
    "No": 2
}

mas_500 = {
    'N' : 1,
    'S' : 0
} 

aglomerado = {
    'Ciudad de Buenos Aires' : 32,
    'Partidos del GBA' : 33

}

ch03 = {
    'Jefe' : 1,
    'Cónyuge/Pareja' : 2,
    'Hijo/Hijastro' : 3,
    'Yerno/Nuera' : 4,
    'Nieto' : 5,
    'Madre/Padre' : 6,
    'Suegro' : 7,
    'Hermano' : 8,
    'Otros familiares' : 9,
    'No familiares' : 10,
}

ch09 = {
    "Sí": 1,
    "No": 2,
    "Menor de 2 años": 3,
    'Ns./Nr.' : 0 
}

ch10 = {
    "Sí, asiste": 1,
    "No asiste, pero asistió": 2,
    "Nunca asistió": 3  
}

In [ ]:
df_combined.loc[:,'CAT_INAC'] = df_combined['CAT_INAC'].replace(cat_inac).astype(int) 
df_combined.loc[:,'CH04'] = df_combined['CH04'].replace(h04).astype(int)
df_combined.loc[:,'CH06'] = df_combined['CH06'].replace(h06).astype(int)
df_combined.loc[:,'CH07'] = df_combined['CH07'].replace(h07).astype(int)
df_combined.loc[:,'CH08'] = df_combined['CH08'].replace(h08).astype(int)
#df_combined.loc[:,'AGLOMERADO'] = df_combined['AGLOMERADO'].map(aglomerado).astype(int)
df_combined.loc[:,'NIVEL_ED'] = df_combined['NIVEL_ED'].replace(nivel_ed).astype(int)
df_combined.loc[:,'ESTADO'] = df_combined['ESTADO'].replace(estado).astype(int)
df_combined['IPCF'] = df_combined['IPCF'].astype(int)

df_combined.loc[:,'COMPONENTE'] = df_combined['COMPONENTE'].replace(componente).astype(int)
df_combined.loc[:,'H15'] = df_combined['H15'].replace(h15).astype(int)
df_combined.loc[:,'MAS_500'] = df_combined['MAS_500'].replace(mas_500).astype(int)
df_combined.loc[:,'AGLOMERADO'] = df_combined['AGLOMERADO'].replace(aglomerado).astype(int)
df_combined.loc[:, 'CH03'] = df_combined['CH03'].replace(ch03).astype(int)
df_combined.loc[:, 'CH09'] = df_combined['CH09'].replace(ch09).astype(int)  # hay 2 valores 9 que no deberían estar
df_combined.loc[:, 'CH10'] = df_combined['CH10'].replace(ch10).astype(int)  # hay 364 valores que son 0 y 2 que son 9 que no deberían ser

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_2740\3711807908.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_combined.loc[:,'CAT_INAC'] = df_combined['CAT_INAC'].replace(cat_inac).astype(int)
C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_2740\3711807908.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_combined.loc[:,'CH04'] = df_combined['CH04'].replace(h04).astype(int)
C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_2740\3711807908.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be remove

In [ ]:
valores_validos = {
    'CH04': [1,2],
    'CH07': [1,2,3,4,5],
    'CH08': [1,2,3,4,9,12,13,23,123],
    'NIVEL_ED': [1,2,3,4,5,6,7,8,9],
    'ESTADO': [0,1,2,3,4],
    'CAT_INAC': [0,1,2,3,4,5,6,7] 
} #'CH06' Y 'IPCF' quedan excluidos porque tiene condiciones de min y max

for col in valores_validos.keys():
    df_combined = df_combined[df_combined[col].isin(valores_validos[col])]

In [ ]:
# Define los límites
maximo_edad = 130 # totalmente arbitrario, pero sirve para sacar extraños muy extraños

# Filtra el DataFrame, manteniendo solo los valores dentro del rango
df_combined = df_combined[(df_combined['IPCF'] >= 0)]
df_combined = df_combined[(df_combined['CH06'] >= 0) & (df_combined['CH06'] <= maximo_edad)]

In [ ]:
df_combined

,CODUSU,NRO_HOGAR,COMPONENTE,H15,ANO4,TRIMESTRE,REGION,MAS_500,AGLOMERADO,PONDERA,...,GDECIFR,PDECIFR,ADECIFR,IPCF,DECCFR,IDECCFR,RDECCFR,GDECCFR,PDECCFR,ADECCFR
0,125098,1.0,1,1,2004,1er. Trimestre,Gran Buenos Aires,0,32,1299.0,...,02,,01,368,07,,06,06,,04
1,125397,1.0,1,1,2004,1er. Trimestre,Gran Buenos Aires,0,32,2018.0,...,00,,00,0,00,,00,00,,00
2,125515,1.0,1,1,2004,1er. Trimestre,Gran Buenos Aires,0,32,1486.0,...,03,,02,440,07,,07,07,,05
3,125558,1.0,1,1,2004,1er. Trimestre,Gran Buenos Aires,0,32,1674.0,...,09,,08,532,08,,08,08,,06
4,125558,1.0,2,1,2004,1er. Trimestre,Gran Buenos Aires,0,32,1674.0,...,09,,08,532,08,,08,08,,06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14693,TQRMNOPUQHKMKNCDEIJAH00861661,1.0,1,1,2024,1,1,0,33,3410.0,...,12.0,NaN,12,0,12,NaN,12,12.0,NaN,12
14694,TQRMNOPUQHKMKNCDEIJAH00861661,1.0,2,2,2024,1,1,0,33,3410.0,...,12.0,NaN,12,0,12,NaN,12,12.0,NaN,12
14695,TQRMNOPUQHKMKNCDEIJAH00861661,1.0,3,2,2024,1,1,0,33,3410.0,...,12.0,NaN,12,0,12,NaN,12,12.0,NaN,12
14696,TQRMNOPUQHKMKNCDEIJAH00861661,1.0,4,2,2024,1,1,0,33,3410.0,...,12.0,NaN,12,0,12,NaN,12,12.0,NaN,12


# 3. MERGE!

In [46]:
hog_2004.columns = hog_2004.columns.str.upper()
hog_2004

,CODUSU,NRO_HOGAR,COMPONENTE,H15,ANO4,TRIMESTRE,REGION,MAS_500,AGLOMERADO,PONDERA,...,DECCFR,IDECCFR,RDECCFR,GDECCFR,PDECCFR,ADECCFR,PJ1_1,PJ2_1,PJ3_1,IDIMPP
0,125098,1.0,1.0,Sí,2004.0,1er. Trimestre,Gran Buenos Aires,S,Ciudad de Buenos Aires,1299.0,...,07,,06,06,,04,0.0,0.0,0.0,00000
1,125397,1.0,1.0,Sí,2004.0,1er. Trimestre,Gran Buenos Aires,S,Ciudad de Buenos Aires,2018.0,...,00,,00,00,,00,0.0,0.0,0.0,00000
2,125515,1.0,1.0,Sí,2004.0,1er. Trimestre,Gran Buenos Aires,S,Ciudad de Buenos Aires,1486.0,...,07,,07,07,,05,0.0,0.0,0.0,00020
3,125558,1.0,1.0,Sí,2004.0,1er. Trimestre,Gran Buenos Aires,S,Ciudad de Buenos Aires,1674.0,...,08,,08,08,,06,0.0,0.0,0.0,00000
4,125558,1.0,2.0,Sí,2004.0,1er. Trimestre,Gran Buenos Aires,S,Ciudad de Buenos Aires,1674.0,...,08,,08,08,,06,0.0,0.0,0.0,00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45284,285733,1.0,1.0,Sí,2004.0,1er. Trimestre,Patagónica,N,Ushuaia - Río Grande,123.0,...,09,09,08,,09,07,0.0,0.0,0.0,00000
45285,285733,1.0,2.0,Sí,2004.0,1er. Trimestre,Patagónica,N,Ushuaia - Río Grande,123.0,...,09,09,08,,09,07,0.0,0.0,0.0,00000
45286,285733,1.0,3.0,Sí,2004.0,1er. Trimestre,Patagónica,N,Ushuaia - Río Grande,123.0,...,09,09,08,,09,07,0.0,0.0,0.0,00000
45287,285733,1.0,4.0,0.0,2004.0,1er. Trimestre,Patagónica,N,Ushuaia - Río Grande,123.0,...,09,09,08,,09,07,0.0,0.0,0.0,00000


In [49]:
#14634 _ind
#61393 _hog
print(14634 +61393 )

76027


In [54]:
# Primero hago el merge de los hogs, no?
df_combined2 = pd.concat([hog_2004, hog_2024], ignore_index=True, sort=False)

# Ahora combino LOS DOS. 
super_combined = pd.merge(df_combined, df_combined2, on=['CODUSU', 'NRO_HOGAR'], how='outer')

In [53]:
super_combined

,CODUSU,NRO_HOGAR,COMPONENTE_x,H15_x,ANO4_x,TRIMESTRE_x,REGION_x,MAS_500_x,AGLOMERADO_x,PONDERA_x,...,IX_TOT,IX_MEN10,IX_MAYEQ10,PONDIH,VII1_1,VII1_2,VII2_1,VII2_2,VII2_3,VII2_4
0,125021,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,125021,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,125021,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,125021,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,125021,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91094,TQVMNORVXHLOTOCDEIJAH00855319,1.0,2,1,2024.0,1,1,0,33,1067.0,...,3.0,0.0,3.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0
91095,TQVMNORVXHLOTOCDEIJAH00855319,1.0,3,1,2024.0,1,1,0,33,1067.0,...,3.0,0.0,3.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0
91096,TQVMNORXUHLLLUCDEFKID00851760,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,1.0,1134.0,1.0,0.0,98.0,0.0,0.0,0.0
91097,TQVMNORXUHLLLUCDEFKID00851760,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,1.0,1927.0,1.0,0.0,98.0,0.0,0.0,0.0


In [23]:
#16104 _2024
#45289 _2004
print (16104+45289)

61393


In [22]:
hog_2004

,CODUSU,NRO_HOGAR,COMPONENTE,H15,ANO4,TRIMESTRE,REGION,MAS_500,AGLOMERADO,PONDERA,...,DECCFR,IDECCFR,RDECCFR,GDECCFR,PDECCFR,ADECCFR,PJ1_1,PJ2_1,PJ3_1,IDIMPP
0,125098,1.0,1.0,Sí,2004.0,1er. Trimestre,Gran Buenos Aires,S,Ciudad de Buenos Aires,1299.0,...,07,,06,06,,04,0.0,0.0,0.0,00000
1,125397,1.0,1.0,Sí,2004.0,1er. Trimestre,Gran Buenos Aires,S,Ciudad de Buenos Aires,2018.0,...,00,,00,00,,00,0.0,0.0,0.0,00000
2,125515,1.0,1.0,Sí,2004.0,1er. Trimestre,Gran Buenos Aires,S,Ciudad de Buenos Aires,1486.0,...,07,,07,07,,05,0.0,0.0,0.0,00020
3,125558,1.0,1.0,Sí,2004.0,1er. Trimestre,Gran Buenos Aires,S,Ciudad de Buenos Aires,1674.0,...,08,,08,08,,06,0.0,0.0,0.0,00000
4,125558,1.0,2.0,Sí,2004.0,1er. Trimestre,Gran Buenos Aires,S,Ciudad de Buenos Aires,1674.0,...,08,,08,08,,06,0.0,0.0,0.0,00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45284,285733,1.0,1.0,Sí,2004.0,1er. Trimestre,Patagónica,N,Ushuaia - Río Grande,123.0,...,09,09,08,,09,07,0.0,0.0,0.0,00000
45285,285733,1.0,2.0,Sí,2004.0,1er. Trimestre,Patagónica,N,Ushuaia - Río Grande,123.0,...,09,09,08,,09,07,0.0,0.0,0.0,00000
45286,285733,1.0,3.0,Sí,2004.0,1er. Trimestre,Patagónica,N,Ushuaia - Río Grande,123.0,...,09,09,08,,09,07,0.0,0.0,0.0,00000
45287,285733,1.0,4.0,0.0,2004.0,1er. Trimestre,Patagónica,N,Ushuaia - Río Grande,123.0,...,09,09,08,,09,07,0.0,0.0,0.0,00000


In [25]:
super_combinado

,CODUSU,NRO_HOGAR,COMPONENTE,H15,ANO4,TRIMESTRE,REGION,MAS_500,AGLOMERADO,PONDERA,...,IX_TOT,IX_MEN10,IX_MAYEQ10,PONDIH,VII1_1,VII1_2,VII2_1,VII2_2,VII2_3,VII2_4
0,125098,1.0,1.0,Sí,2004.0,1er. Trimestre,Gran Buenos Aires,S,Ciudad de Buenos Aires,1299.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,125397,1.0,1.0,Sí,2004.0,1er. Trimestre,Gran Buenos Aires,S,Ciudad de Buenos Aires,2018.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,125515,1.0,1.0,Sí,2004.0,1er. Trimestre,Gran Buenos Aires,S,Ciudad de Buenos Aires,1486.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,125558,1.0,1.0,Sí,2004.0,1er. Trimestre,Gran Buenos Aires,S,Ciudad de Buenos Aires,1674.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,125558,1.0,2.0,Sí,2004.0,1er. Trimestre,Gran Buenos Aires,S,Ciudad de Buenos Aires,1674.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61388,TQRMNOPXXHLKKUCDEFNFF00852324,1.0,NaN,NaN,2024.0,1,41,N,7,336.0,...,3.0,0.0,3.0,0.0,96.0,0.0,2.0,3.0,0.0,0.0
61389,TQRMNOPQTHKMLMCDEHMHF00861771,1.0,NaN,NaN,2024.0,1,42,N,26,141.0,...,2.0,0.0,2.0,156.0,97.0,0.0,1.0,2.0,0.0,0.0
61390,TQRMNOSYXHMMLNCDEFNFF00798999,1.0,NaN,NaN,2024.0,1,41,N,7,305.0,...,6.0,1.0,5.0,353.0,2.0,0.0,1.0,3.0,4.0,0.0
61391,TQRMNOPTSHLOLQCDEFNFF00852331,1.0,NaN,NaN,2024.0,1,41,N,7,331.0,...,1.0,0.0,1.0,420.0,97.0,0.0,98.0,0.0,0.0,0.0
